In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys

from sklearn.model_selection import train_test_split
sys.path.append('../../Share/')
import Model

def random_downsample_Num(X, y=None, n_keep=None, random_state=42):
    np.random.seed(random_state)
    n_samples = X.shape[0]

    if n_keep is None or n_keep > n_samples:
        raise ValueError(f"n_keep must be between 1 and {n_samples}, got {n_keep}")

    idx_keep = np.random.choice(n_samples, size=n_keep, replace=False)
    X_down = X[idx_keep]

    if y is not None:
        y_down = y[idx_keep]
        return X_down, y_down
    else:
        return X_down

### Subject 1 버리기 + use only subject 2 and 3

In [2]:
path = 'C:/Users/hml76/Desktop/Jupyter/Federated Prototype Learning/Dataset/Ours_cleaned/'
fname = os.listdir(path)

subjects = {
    "subject1": {"X": [], "Y": []},
    "subject2": {"X": [], "Y": []},
    "subject3": {"X": [], "Y": []},
}

for f in fname:
    for i in range(10):
        X = np.array(pd.read_csv(path + f'{f}/rep{i}_X.csv'))
        y = np.array(pd.read_csv(path + f'{f}/rep{i}_Y.csv'))

        # shuffle
        #indices = np.random.permutation(len(X))
        #X, y = X[indices], y[indices]

        # convert one-hot to integer labels
        y_int = np.argmax(y, axis=1)
        num_classes = np.max(y_int) + 1

        # example rule to assign data to subject dict
        if "subject1" in f:
            subjects["subject1"]["X"].append(X)
            subjects["subject1"]["Y"].append(y_int)
        elif "subject2" in f:
            subjects["subject2"]["X"].append(X)
            subjects["subject2"]["Y"].append(y_int)
        elif "subject3" in f:
            subjects["subject3"]["X"].append(X)
            subjects["subject3"]["Y"].append(y_int)

1. K1/4/7/10 initialization (저걸로 학습시키기) + Test on rest of the subjects
2.

# Subject 2

In [ ]:
for t in range(num_trials):
    X, y = X_lst[t].reshape(-1,16,14,1), y_lst[t]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
    feature_set = 14
    print(pd.Series(y_train).value_counts())

In [22]:
# Baseline training on subjects
X_lst, y_lst = subjects["subject2"]["X"], subjects["subject2"]["Y"]

def feature_analysis(X_lst, y_lst):
    num_trials = len(X_lst)
    all_acc = []
    for t in range(num_trials):
        X, y = X_lst[t].reshape(-1,16,14,1), y_lst[t]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
        feature_set = 14
        feat_acc = []

        for f in range(feature_set):
            X_train_f = X_train[:,:,f,:]
            X_test_f = X_test[:,:,f,:]
            #model = Model.Original_model(input_shape=X_train.shape[1:], num_class=6)
            model = Model.Original_model_1DCNN(input_size=X_train_f.shape[1:], num_class=6)

            history, _ = Model.Train_model(model, X_train_f, y_train, X_test_f, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)
            #plt.plot(history.history['val_accuracy'])
            #plt.plot(history.history['accuracy'])
            #plt.show()

            print(f"\tTrial {t+1}/{num_trials}/feat{f} | Val Acc: {np.max(history.history['val_accuracy'])}")
            feat_acc.append(np.max(history.history['val_accuracy']))
        all_acc.append(feat_acc)

    return all_acc

In [26]:
X_lst2, y_lst2 = subjects["subject2"]["X"], subjects["subject2"]["Y"]
all_acc_S2 = feature_analysis(X_lst, y_lst)
#np.mean(all_acc, axis=1)
#[0.54582531, 0.59289471, 0.71110999, 0.68362534, 0.76991238, 0.49386236, 0.49553254, 0.48481364, 0.5051527 , 0.39519543]# = ave - subject 2

In [28]:
X_lst3, y_lst3 = subjects["subject3"]["X"], subjects["subject3"]["Y"]
all_acc_S3 = feature_analysis(X_lst, y_lst)
#[0.58912348, 0.59857687, 0.67890562, 0.50030979, 0.45444015, 0.53214064, 0.57473667, 0.62291515, 0.49060753, 0.47509458]

In [25]:
np.mean(all_acc_S3, axis=1)

array([0.58912348, 0.59857687, 0.67890562, 0.50030979, 0.45444015,
       0.53214064, 0.57473667, 0.62291515, 0.49060753, 0.47509458])

# Inter-session

In [17]:
def inter_session_run():
    X_lst, y_lst = subjects["subject2"]["X"], subjects["subject2"]["Y"]
    num_trials = len(X_lst)
    all_acc_K = []

    for t in [1, 4, 7, 9]:
        # Train split
        X_train = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t)]
        X_train = np.concatenate(X_train, axis=0)

        y_train = [y_lst[i] for i in range(t)]
        y_train = np.concatenate(y_train, axis=0)

        # Test split
        X_test = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t, len(X_lst))]
        X_test = np.concatenate(X_test, axis=0)

        y_test = [y_lst[i] for i in range(t, len(y_lst))]
        y_test = np.concatenate(y_test, axis=0)

        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

        f = 4
        X_train_f, X_test_f = X_train[:,:,f,:], X_test[:,:,f,:]
        X_test_f, y_test = random_downsample_Num(X_test_f, y_test, n_keep=1000, random_state=42)

        model = Model.Original_model_1DCNN(input_size=X_train_f.shape[1:], num_class=6)
        history, _ = Model.Train_model(model, X_train_f, y_train, X_test_f, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

        print(f"\tTrial {t+1}/{num_trials}/feat{f} | Val Acc: {np.max(history.history['val_accuracy'])}")
        all_acc_K.append(np.max(history.history['val_accuracy']))

(6303, 16, 14, 1) (6303,) (64036, 16, 14, 1) (64036,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 51.94%
Maximum validation accuracy : 32.1%
	Trial 2/10/feat4 | Val Acc: 0.32100000977516174
(26399, 16, 14, 1) (26399,) (43940, 16, 14, 1) (43940,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 58.16%
Maximum validation accuracy : 29.8%
	Trial 5/10/feat4 | Val Acc: 0.2980000078678131
(48077, 16, 14, 1) (48077,) (22262, 16, 14, 1) (22262,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 53.06%
Maximum validation accuracy : 36.4%
	Trial 8/10/feat4 | Val Acc: 0.36399999260902405
(63084, 16, 14, 1) (63084,) (7255, 16, 14, 1) (7255,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 49.43%
Maximum validation accuracy : 37.8%
	Trial 10/10/feat4 | Val Acc: 0.37799999117851

In [18]:
all_acc_K

[np.float64(0.32100000977516174),
 np.float64(0.2980000078678131),
 np.float64(0.36399999260902405),
 np.float64(0.3779999911785126)]

In [20]:
# Baseline training on subject2
X_lst, y_lst = subjects["subject2"]["X"], subjects["subject2"]["Y"]
num_trials = len(X_lst)
all_acc_K = []

for t in [1, 4, 7, 9]:
    X_train = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t)]
    X_train = np.concatenate(X_train, axis=0)
    y_train = [y_lst[i] for i in range(t)]
    y_train = np.concatenate(y_train, axis=0)

    X_test = [X_lst[i].reshape(-1, 16, 14, 1) for i in range(t, len(X_lst))]
    X_test = np.concatenate(X_test, axis=0)
    y_test = [y_lst[i] for i in range(t, len(y_lst))]
    y_test = np.concatenate(y_test, axis=0)

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    #f = 4
    #X_train_f, X_test_f = X_train[:,:,f,:], X_test[:,:,f,:]
    X_test, y_test = random_downsample_Num(X_test, y_test, n_keep=1000, random_state=42)

    model = Model.Original_model(input_shape=X_train.shape[1:], num_class=6)
    history, _ = Model.Train_model(model, X_train, y_train, X_test, y_test, set_epoch=50, set_batch_size=128, Model_name=None, set_verbose=False, save_model_set=False)

    print(f"\tTrial {t+1}/{num_trials}/feat{f} | Val Acc: {np.max(history.history['val_accuracy'])}")
    all_acc_K.append(np.max(history.history['val_accuracy']))
all_acc_K

(6303, 16, 14, 1) (6303,) (64036, 16, 14, 1) (64036,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 53.43%
Maximum validation accuracy : 22.5%
	Trial 2/10/feat4 | Val Acc: 0.22499999403953552
(26399, 16, 14, 1) (26399,) (43940, 16, 14, 1) (43940,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 59.92%
Maximum validation accuracy : 24.3%
	Trial 5/10/feat4 | Val Acc: 0.24300000071525574
(48077, 16, 14, 1) (48077,) (22262, 16, 14, 1) (22262,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 50.81%
Maximum validation accuracy : 34.3%
	Trial 8/10/feat4 | Val Acc: 0.34299999475479126
(63084, 16, 14, 1) (63084,) (7255, 16, 14, 1) (7255,)
Start Training (total epochs: 50)...
Finish Training! (Model is NOT saved)

Maximum training accuracy : 49.41%
Maximum validation accuracy : 34.6%
	Trial 10/10/feat4 | Val Acc: 0.3459999859333

[np.float64(0.22499999403953552),
 np.float64(0.24300000071525574),
 np.float64(0.34299999475479126),
 np.float64(0.34599998593330383)]

In [21]:
all_acc_K

[np.float64(0.22499999403953552),
 np.float64(0.24300000071525574),
 np.float64(0.34299999475479126),
 np.float64(0.34599998593330383)]